In [351]:
# columns to keep as input features in the dataset
static_columns = ['gender', 'ethnicity', 'age', 'insurance', 'admission_type', 'first_careunit', 'icd9_codes']

#icd9 codes column name
icd9 = 'icd9_codes'

# prediciton label column in the dataset
target = 'mort_hosp'

### Loading the dataset from file

In [352]:
%load_ext autoreload
from __future__ import print_function, division

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [353]:
%autoreload

import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

In [354]:
%%time
d1 = pd.read_hdf('../Mortality/output2/C.h5')
d3 = pd.read_hdf('../Mortality/output2/all_hourly_data.h5','patients')

CPU times: user 197 ms, sys: 13.1 ms, total: 210 ms
Wall time: 224 ms


In [355]:
pd.set_option('display.max_columns', None)

In [356]:
d7 = pd.merge(d3,d1,how='right',on=['subject_id','hadm_id','icustay_id'])

In [357]:
d1.head()

,,,icd9_codes
subject_id,hadm_id,icustay_id,
27513,163557,200003,"[0389, 70709, 5761, 9974, 99591, V1009]"
20707,129310,200007,"[41011, 9971, 42731, 41401, 4019, 2720, V173]"
29904,129607,200009,"[4240, 99811, 9998, 4019, 2859, V173]"
9514,127229,200014,"[4414, 41091, 4280, 49121, 53541, 5307, 41401,..."
21789,112486,200019,"[43411, 5070, 3485, 4019, 2720, 412, V707]"


In [358]:
d7.head()

,,,gender,ethnicity,age,insurance,admittime,diagnosis_at_admission,dischtime,discharge_location,fullcode_first,dnr_first,fullcode,dnr,dnr_first_charttime,timecmo_chart,cmo_first,cmo_last,cmo,deathtime,intime,outtime,los_icu,admission_type,first_careunit,mort_icu,mort_hosp,hospital_expire_flag,hospstay_seq,readmission_30,max_hours,icd9_codes
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,2101-10-20 19:08:00,HYPOTENSION,2101-10-31 13:58:00,SNF,1.0,0.0,1.0,1.0,2101-10-20 22:00:00,NaT,0.0,0.0,0.0,NaT,2101-10-20 19:10:11,2101-10-26 20:43:09,6.064560,EMERGENCY,MICU,0.0,0.0,0.0,1.0,0.0,145.0,"[0389, 78559, 5849, 4275, 41071, 4280, 6826, 4..."
4,185777,294638,F,WHITE,47.845047,Private,2191-03-16 00:28:00,"FEVER,DEHYDRATION,FAILURE TO THRIVE",2191-03-23 18:41:00,HOME WITH HOME IV PROVIDR,1.0,0.0,1.0,0.0,NaT,NaT,0.0,0.0,0.0,NaT,2191-03-16 00:29:31,2191-03-17 16:46:31,1.678472,EMERGENCY,MICU,0.0,0.0,0.0,1.0,0.0,40.0,"[042, 1363, 7994, 2763, 7907, 5715, 04111, V09..."
6,107064,228232,F,WHITE,65.942297,Medicare,2175-05-30 07:15:00,CHRONIC RENAL FAILURE/SDA,2175-06-15 16:00:00,HOME HEALTH CARE,1.0,0.0,1.0,0.0,NaT,NaT,0.0,0.0,0.0,NaT,2175-05-30 21:30:54,2175-06-03 13:39:54,3.672917,ELECTIVE,SICU,0.0,0.0,0.0,1.0,0.0,88.0,"[40391, 4440, 9972, 2766, 2767, 2859, 2753, V1..."
9,150750,220597,M,UNKNOWN/NOT SPECIFIED,41.790228,Medicaid,2149-11-09 13:06:00,HEMORRHAGIC CVA,2149-11-14 10:15:00,DEAD/EXPIRED,1.0,0.0,1.0,0.0,NaT,NaT,0.0,0.0,0.0,2149-11-14 10:15:00,2149-11-09 13:07:02,2149-11-14 20:52:14,5.323056,EMERGENCY,MICU,1.0,1.0,1.0,1.0,0.0,127.0,"[431, 5070, 4280, 5849, 2765, 4019]"
11,194540,229441,F,WHITE,50.148295,Private,2178-04-16 06:18:00,BRAIN MASS,2178-05-11 19:00:00,HOME HEALTH CARE,1.0,0.0,1.0,0.0,NaT,NaT,0.0,0.0,0.0,NaT,2178-04-16 06:19:32,2178-04-17 20:21:05,1.584410,EMERGENCY,SICU,0.0,0.0,0.0,1.0,0.0,38.0,[1913]


In [359]:
d7.columns

Index(['gender', 'ethnicity', 'age', 'insurance', 'admittime',
       'diagnosis_at_admission', 'dischtime', 'discharge_location',
       'fullcode_first', 'dnr_first', 'fullcode', 'dnr', 'dnr_first_charttime',
       'timecmo_chart', 'cmo_first', 'cmo_last', 'cmo', 'deathtime', 'intime',
       'outtime', 'los_icu', 'admission_type', 'first_careunit', 'mort_icu',
       'mort_hosp', 'hospital_expire_flag', 'hospstay_seq', 'readmission_30',
       'max_hours', 'icd9_codes'],
      dtype='object')

In [360]:
# Dropping all the columns except for the ones required for the model - 
# static features like gender, age, ethnicity, insurance, first_careunit and admission_type. 
# Also keep the label columns like mort_icu, mort_hosp
d7 = d7[static_columns+[target]]
#d7.drop(columns=['deathtime','intime','outtime','cmo','admittime','diagnosis_at_admission', 'dischtime','fullcode_first', 'dnr_first', 'fullcode', 'dnr', 'dnr_first_charttime','timecmo_chart', 'cmo_first', 'cmo_last'],inplace=True)

In [361]:
# check the unique for each static column and make them in categories
# use a category name for NaN or null values
print(d7.gender.unique())
print(d7.ethnicity.unique())
print(d7.insurance.unique())
print(d7.admission_type.unique())
print(d7.first_careunit.unique())

[M, F, NaN]
Categories (2, object): [M, F]
[WHITE, UNKNOWN/NOT SPECIFIED, PATIENT DECLINED TO ANSWER, BLACK/AFRICAN AMERICAN, ASIAN, ..., HISPANIC/LATINO - MEXICAN, ASIAN - KOREAN, AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN..., ASIAN - JAPANESE, NaN]
Length: 42
Categories (41, object): [WHITE, UNKNOWN/NOT SPECIFIED, PATIENT DECLINED TO ANSWER, BLACK/AFRICAN AMERICAN, ..., HISPANIC/LATINO - MEXICAN, ASIAN - KOREAN, AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN..., ASIAN - JAPANESE]
['Medicare' 'Private' 'Medicaid' 'Government' 'Self Pay' nan]
[EMERGENCY, ELECTIVE, URGENT, NaN]
Categories (3, object): [EMERGENCY, ELECTIVE, URGENT]
[MICU, SICU, CCU, CSRU, TSICU, NaN]
Categories (5, object): [MICU, SICU, CCU, CSRU, TSICU]


In [362]:
d7.ethnicity = d7.ethnicity.astype('str')
d7.ethnicity.replace('nan','ethncity_nan',inplace=True)
d7.gender = d7.gender.astype('str')
d7.gender.replace('nan','gender_nan',inplace=True)
d7.insurance = d7.insurance.astype('str')
d7.insurance.replace('nan','insurance_nan',inplace=True)
d7.admission_type = d7.admission_type.astype('str')
d7.admission_type.replace('nan','admission_type_nan',inplace=True)
d7.first_careunit = d7.first_careunit.astype('str')
d7.first_careunit.replace('nan','ethncity_nan',inplace=True)

In [363]:
d7.ethnicity = d7.ethnicity.astype('str')
d7.loc[d7['ethnicity'].str.contains('WHITE'), 'ethnicity']    = 'WHITE'
d7.loc[d7['ethnicity'].str.contains('BLACK'), 'ethnicity']    = 'BLACK/AFRICAN AMERICAN'
d7.loc[d7['ethnicity'].str.contains('ASIAN'), 'ethnicity']    = 'ASIAN'
d7.loc[d7['ethnicity'].str.contains('HISPANIC'), 'ethnicity'] = 'HISPANIC/LATINO'
d7.loc[d7['ethnicity'].str.contains('DECLINED'), 'ethnicity'] = 'OTHER/UNKNOWN'
d7.loc[d7['ethnicity'].str.contains('MULTI'), 'ethnicity']    = 'OTHER/UNKNOWN'
d7.loc[d7['ethnicity'].str.contains('UNKNOWN'), 'ethnicity']  = 'OTHER/UNKNOWN'
d7.loc[d7['ethnicity'].str.contains('OTHER'), 'ethnicity']  = 'OTHER/UNKNOWN'
d7.loc[d7['ethnicity'].str.contains('ethncity_nan'), 'ethnicity']  = 'OTHER/UNKNOWN'
d7.loc[d7['ethnicity'].str.contains('UNABLE TO OBTAIN'), 'ethnicity']  = 'OTHER/UNKNOWN'

/Users/rishabhkhanna/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:2877: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


In [364]:
print(d7.gender.unique())
print(d7.ethnicity.unique())
print(d7.insurance.unique())
print(d7.admission_type.unique())
print(d7.first_careunit.unique())

['M' 'F' 'gender_nan']
['WHITE' 'OTHER/UNKNOWN' 'BLACK/AFRICAN AMERICAN' 'ASIAN'
 'HISPANIC/LATINO' 'AMERICAN INDIAN/ALASKA NATIVE' 'PORTUGUESE'
 'CARIBBEAN ISLAND' 'SOUTH AMERICAN' 'MIDDLE EASTERN'
 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE']
['Medicare' 'Private' 'Medicaid' 'Government' 'Self Pay' 'insurance_nan']
['EMERGENCY' 'ELECTIVE' 'URGENT' 'admission_type_nan']
['MICU' 'SICU' 'CCU' 'CSRU' 'TSICU' 'ethncity_nan']


### Constructing Dictionaries, preparing labels and replacing the categorical values with dictionary integer values

In [365]:
# removing the null labeled records
d7 = d7[d7[target].notna()]

In [366]:
# keep only the static columns
static = d7[[i for i in static_columns if i != icd9]]
#static = d7.drop(columns=['age','los_icu','mort_icu','mort_hosp','hospital_expire_flag','hospstay_seq','readmission_30','max_hours','icd9_codes','discharge_location'])

In [367]:
static.head()

,,,gender,ethnicity,age,insurance,admission_type,first_careunit
subject_id,hadm_id,icustay_id,,,,,,
3,145834,211552,M,WHITE,76.526792,Medicare,EMERGENCY,MICU
4,185777,294638,F,WHITE,47.845047,Private,EMERGENCY,MICU
6,107064,228232,F,WHITE,65.942297,Medicare,ELECTIVE,SICU
9,150750,220597,M,OTHER/UNKNOWN,41.790228,Medicaid,EMERGENCY,MICU
11,194540,229441,F,WHITE,50.148295,Private,EMERGENCY,SICU


In [368]:
# convert static columns into unique keys for dictionary
static_dict_keys = list(static.first_careunit.unique())+list(static.gender.unique())+list(static.insurance.unique())+list(static.admission_type.unique())+list(static.ethnicity.unique())

In [369]:
static_dict = {}
counter = 1
for k in static_dict_keys:
    static_dict[k] = counter
    counter += 1

In [370]:
d7.insurance = d7.insurance.astype('str')
for k in list(static.insurance.unique()):
    d7.loc[d7['insurance'].str.contains(k,na=False), 'insurance']    = static_dict[k]

In [371]:
d7.admission_type = d7.admission_type.astype('str')
for k in list(static.admission_type.unique()):
    d7.loc[d7['admission_type'].str.contains(k,na=False), 'admission_type']    = static_dict[k]

In [372]:
d7.first_careunit = d7.first_careunit.astype('str')
for k in list(static.first_careunit.unique()):
    d7.loc[d7['first_careunit'].str.contains(k,na=False), 'first_careunit']    = static_dict[k]

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [373]:
d7.ethnicity = d7.ethnicity.astype('str')
for k in list(static.ethnicity.unique()):
    d7.loc[d7['ethnicity'].str.contains(k,na=False), 'ethnicity']    = static_dict[k]

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [374]:
d7.gender = d7.gender.astype('str')
for k in list(static.gender.unique()):
    d7.loc[d7['gender'].str.contains(k,na=False), 'gender']    = static_dict[k]

In [375]:
# keep only the static columns
static = d7[[i for i in static_columns if i != icd9]]
#static = d7.drop(columns=['age','los_icu','mort_icu','mort_hosp','hospital_expire_flag','hospstay_seq','readmission_30','max_hours','icd9_codes','discharge_location'])

In [376]:
static.head()

,,,gender,ethnicity,age,insurance,admission_type,first_careunit
subject_id,hadm_id,icustay_id,,,,,,
3,145834,211552,6,16,76.526792,8,13,1
4,185777,294638,7,16,47.845047,9,13,1
6,107064,228232,7,16,65.942297,8,14,2
9,150750,220597,6,17,41.790228,10,13,1
11,194540,229441,7,16,50.148295,9,13,2


In [377]:
icd9_code_dict = {}
for k in np.concatenate(d7[icd9].to_numpy()):
    if k not in icd9_code_dict:
        icd9_code_dict[k] = counter
        counter += 1

In [378]:
len(icd9_code_dict)

6040

In [379]:
class dummy:
    max_codes = 0
    def get_max(self,a):
        self.max_codes = max(self.max_codes, len(a))
        return a
def get_val(a,max_codes):
    for i in range(len(a)):
        a[i] = icd9_code_dict[str(a[i])]
    a = a + [0]*(max_codes-len(a))
    return a

In [380]:
# flattening the list of icd9 codes and replacing the values in dataset with integer values from dictionary
obj = dummy()
d7[icd9] = d7[icd9].apply(lambda x: obj.get_max(x))
d7[icd9] = d7[icd9].apply(lambda x: (get_val(x,obj.max_codes),x))

In [381]:
d7[icd9] = d7[icd9].apply(lambda x: x[0])

In [382]:
column_names = ['Code'+str(i) for i in range(obj.max_codes)]

In [383]:
d7[column_names] = d7[icd9].apply(pd.Series)

In [384]:
# diving the age into buckets of size 10 years
bin = np.array([-1, 20, 30, 40, 50, 60, 70, 80, np.Inf])
label = ['AGE1', 'AGE2', 'AGE3', 'AGE4', 'AGE5', 'AGE6', 'AGE7', 'AGE8']
d7['age'] = pd.cut(d7['age'], bins=bin, labels=label)

In [385]:
age_dict = {}
for k in label:
    age_dict[k] = counter
    counter += 1

In [386]:
d7.age = d7.age.astype('str')
for k in label:
    d7.loc[d7['age'] == k, 'age']    = age_dict[k]

In [387]:
static = d7[[i for i in static_columns+column_names+[target] if i != icd9]]

In [388]:
static.head()

,,,gender,ethnicity,age,insurance,admission_type,first_careunit,Code0,Code1,Code2,Code3,Code4,Code5,Code6,Code7,Code8,Code9,Code10,Code11,Code12,Code13,Code14,Code15,Code16,Code17,Code18,Code19,Code20,Code21,Code22,Code23,Code24,Code25,Code26,Code27,Code28,Code29,Code30,Code31,Code32,Code33,Code34,Code35,Code36,Code37,Code38,mort_hosp
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,6,16,6073,8,13,1,27,28,29,30,31,32,33,34,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
4,185777,294638,7,16,6070,9,13,1,36,37,38,39,40,41,42,43,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
6,107064,228232,7,16,6072,8,14,2,45,46,47,48,49,50,51,52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0
9,150750,220597,6,17,6070,10,13,1,53,54,32,29,55,56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
11,194540,229441,7,16,6071,9,13,2,57,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [389]:
print(min(static_dict.values()),max(static_dict.values()))

1 26


In [390]:
print(min(icd9_code_dict.values()),max(icd9_code_dict.values()))

27 6066


In [391]:
print(min(age_dict.values()),max(age_dict.values()))

6067 6074


### Training Model

In [406]:
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, classification_report

import torch, torch.utils.data as utils, torch.nn as nn, torch.nn.functional as F, torch.optim as optim
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from tqdm import tqdm
from time import time

In [407]:
full_dataset = static.copy(deep=True)

In [408]:
# use this when the dataset needs to be divided into three sets without external files for subject ids
train_size = int(0.7 * len(full_dataset))
valid_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - valid_size
trainX, trainY = full_dataset[:train_size].drop(columns=[target]), full_dataset[:train_size][target]
validX, validY = full_dataset[train_size:train_size+valid_size].drop(columns=[target]), full_dataset[train_size:train_size+valid_size][target]
testX, testY = full_dataset[train_size+valid_size:].drop(columns=[target]), full_dataset[train_size+valid_size:][target]

In [409]:
# using subject ids from a file for diving the dataset
# use this logic only if the subject ids are available in a seperate file
# otherwise divide the dataset into ratios later
train_df = pd.read_csv('train_subj.csv',names=['subject_id'])
dev_df = pd.read_csv('dev_subj.csv',names=['subject_id'])
test_df = pd.read_csv('test_subj.csv',names=['subject_id'])

# use this when a seperate file for diving the dataset using external files for subject ids
train = pd.merge(full_dataset,train_df,how='right',on=['subject_id'])
valid = pd.merge(full_dataset,dev_df,how='right',on=['subject_id'])
test = pd.merge(full_dataset,test_df,how='right',on=['subject_id'])

trainX, trainY = train[[i for i in static_columns+column_names if i != icd9]], train[target]
validX, validY = valid[[i for i in static_columns+column_names if i != icd9]], valid[target]
testX, testY = test[[i for i in static_columns+column_names if i != icd9]], test[target]

In [410]:
len(testX)+len(trainX)+len(validX)

23944

In [411]:
trainY.value_counts()

0.0    14934
1.0     1826
Name: mort_hosp, dtype: int64

In [412]:
def prepare_dataloader(X, label, batch_size, shuffle=True):
    pos_weight = torch.tensor((len(label) - np.sum(label))/np.sum(label))
    X     = torch.from_numpy(X.values.astype(np.int))
    label = torch.from_numpy(label.values.astype(np.float64))
    dataset = utils.TensorDataset(X, label)

    
#     pos_weight = torch.tensor((len(label) - torch.sum(label))/torch.sum(label))
    return pos_weight, utils.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last = True)
#     return (pos_weight, utils.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last = True))

In [413]:
class CBOW(nn.Module):
    def __init__(self, num_codes):
        super(CBOW, self).__init__()
        self.dropout = nn.Dropout(p=0.5)
        # Embedding dimensions
        self.embed_dim = 21

        # Embedding layers
        self.embed_layer = nn.Embedding(num_embeddings=num_codes, embedding_dim=self.embed_dim, padding_idx=0)
        
        # FC layers
        self.fc_layer = nn.Linear(45*self.embed_dim, 32)
        self.fc_all = nn.Linear(32, 1)
    
    def forward(self, codes):
        embed = self.embed_layer(codes)
        embed = embed.view(-1,45*(self.embed_dim))
        out = F.leaky_relu(self.fc_layer(embed))
        out = F.sigmoid(self.fc_all(out))
        return out

In [414]:
net = CBOW(6076)

In [415]:
pos_weight, train_loader = prepare_dataloader(trainX, trainY, 64)
_, valid_loader = prepare_dataloader(validX, validY, 64)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(net.parameters(), lr = 0.001,weight_decay= 0.001)
epoch_times = []

In [416]:
from __future__ import print_function
import torch
import numpy as np
import pandas as pd
import pickle
import scipy.stats as st
import os
from tqdm import tqdm
from sklearn.metrics import *
from sklearn.calibration import calibration_curve
from pdb import set_trace as bp

for epoch in range(100):
    print('-----------------------------------------')
    print('Epoch: {}'.format(epoch))
    net.train()
    time_start = time()
    loss_val = 0
    for i, (X, label) in enumerate(tqdm(train_loader),0):
        optimizer.zero_grad()
        label_pred = net(X)
        loss = criterion(label_pred, label.view(-1,1))
        loss.backward()
        optimizer.step()
        loss_val += loss.item()
    
    time_end = time()
    epoch_times.append(time_end-time_start)
    print('Loss is: ', loss_val/(i+1))
    if epoch%5 == 0:
        _, valid_loader = prepare_dataloader(validX, validY, 64)
        net.eval()
        label_pred = torch.Tensor([])
        label_test = torch.Tensor([])
        with torch.no_grad():
            for i, (X, label) in enumerate(valid_loader, 0):
                # move to GPU if available

                label_pred_batch = net(X)
                label_pred_batch.apply_(lambda x: 1 if x>0.5 else 0)
                label_pred = torch.cat((label_pred, label_pred_batch.cpu()))
                label_test = torch.cat((label_test, label.view(-1,1)))


        # Average precision
        avpre = average_precision_score(label_test, label_pred)

        # Determine AUROC score
        auroc = roc_auc_score(label_test, label_pred)

        acc   = accuracy_score(label_test, label_pred)
        
        f1 = f1_score(label_test,label_pred)
        print(f1)
        print(classification_report(label_test,label_pred))

  0%|          | 0/261 [00:00<?, ?it/s]/Users/rishabhkhanna/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  7%|▋         | 17/261 [00:00<00:01, 162.14it/s]

-----------------------------------------
Epoch: 0


  0%|          | 0/261 [00:00<?, ?it/s]/Users/rishabhkhanna/miniconda3/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
  9%|▉         | 24/261 [00:00<00:01, 232.36it/s]

Loss is:  1.192133938047945
0.330757341576507
              precision    recall  f1-score   support

         0.0       0.94      0.85      0.89      2148
         1.0       0.25      0.49      0.33       220

    accuracy                           0.82      2368
   macro avg       0.60      0.67      0.61      2368
weighted avg       0.88      0.82      0.84      2368

-----------------------------------------
Epoch: 1


  8%|▊         | 22/261 [00:00<00:01, 214.93it/s]

...

Loss is:  0.9432060534925231
-----------------------------------------
Epoch: 98


  9%|▉         | 23/261 [00:00<00:01, 228.04it/s]

Loss is:  0.9440720657676304
-----------------------------------------
Epoch: 99


100%|██████████| 261/261 [00:01<00:00, 225.93it/s]

Loss is:  0.9437819097370682


In [417]:
_, test_loader = prepare_dataloader(testX, testY, 64)
net.eval()
label_pred = torch.Tensor([])
label_test = torch.Tensor([])
with torch.no_grad():
  for i, (X, label) in enumerate(test_loader, 0):
    # move to GPU if available

    label_pred_batch = net(X)
    label_pred_batch.apply_(lambda x: 1 if x>0.5 else 0)
    label_pred = torch.cat((label_pred, label_pred_batch.cpu()))
    label_test = torch.cat((label_test, label.view(-1,1)))


# Average precision
avpre = average_precision_score(label_test, label_pred)

# Determine AUROC score
auroc = roc_auc_score(label_test, label_pred)

acc   = accuracy_score(label_test, label_pred)

f1 = f1_score(label_test,label_pred)
print('F1 Score : ',f1)
print(classification_report(label_test,label_pred))

F1 Score :  0.5592592592592592
              precision    recall  f1-score   support

         0.0       0.96      0.93      0.94      4249
         1.0       0.51      0.62      0.56       487

    accuracy                           0.90      4736
   macro avg       0.73      0.78      0.75      4736
weighted avg       0.91      0.90      0.90      4736

